<div style="text-align: center;background-color: gold; padding: 25px; border-radius: 5px; margin-bottom: 10px;">
    <h2> BUY (Y/N) PREDICTION USING MACHINE LEARNING AND Statistical Approach.</h2>
</div>

* Main Idea : `Robi Datathon 3.0 Problem 3` 😊✨
* Coding    : `Shaikh Hossain Elin` 🏆

# Dataset Description
For this competition,
The MAGIC_KEY id is the unique identifier and the rest columns are the features.

You need to predict which of the Magic Keys given in “problem 1.csv” will buy milk and/or meat in the first 15 days of March-2019. Put Y in the purchase column if the Magic Keys will purchase and N if the Magic Keys will not make a purchase. Follow "sample submission 1.csv" as the template.
## Files

    boxes.csv - contains box informations
    purchase.csv - contains a product purchase info
    problem 2.csv - the test set
    sample submission 2.csv - a sample submission file in the correct format

<div style="text-align: center; background-color: orange; padding: 20px;">
    <h3> <a href="https://github.com/oitik/Robi-Datathon-3.0/blob/master/robidatathon-problem1.ipynb">Click --> Alternate Notebook BY TANBEER JUBAER</a></h3>
    
</div>


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

%matplotlib inline

from tqdm import tqdm
import string
import statistics
import re
import joblib
import random
import statistics
from collections import Counter
from scipy import stats
import os

# import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import zipfile
from wordcloud import WordCloud

In [5]:
box = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-1/boxes.csv')
box

,BOX_ID,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,1,Premium,Home Delivery - CoD,0.0,2.7,9.96
1,2,Premium,Home Delivery - CoD,0.0,2.3,11.96
2,3,Premium,Home Delivery - CoD,0.0,2.4,11.96
3,4,Premium,Home Delivery - CoD,0.0,2.5,11.96
4,5,Premium,Home Delivery - CoD,0.0,2.6,11.96
...,...,...,...,...,...,...
285,286,Standard,Home Delivery - Digital Payment,0.0,4.0,15.96
286,287,Standard,Home Delivery - Digital Payment,0.0,2.9,15.96
287,288,Standard,Home Delivery - Digital Payment,0.0,3.6,15.96
288,289,Standard,Home Delivery - Digital Payment,0.0,4.7,15.96


In [6]:
box.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BOX_ID           290 non-null    int64  
 1   QUALITY          290 non-null    object 
 2   DELIVERY_OPTION  290 non-null    object 
 3   MILK             290 non-null    float64
 4   MEAT             290 non-null    float64
 5   UNIT_PRICE       290 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 13.7+ KB


In [7]:
purchase = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-1/purchase.csv')
purchase

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT
0,1/2/2019,2CED678A247,12.0,1.0
1,1/2/2019,2BF58D91BA1,12.0,1.0
2,1/2/2019,2C15B86534E,99.0,1.0
3,1/2/2019,2C32D9A859A,6.0,1.0
4,1/2/2019,2C7A55404D1,4.0,1.0
...,...,...,...,...
2455859,28/10/2018,2BD992B5538,12.0,1.0
2455860,28/10/2018,2C97CD72233,17.0,1.0
2455861,28/10/2018,2C91C61D372,40.0,1.0
2455862,28/10/2018,2CD70CFC4E3,51.0,1.0


In [8]:
purchase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2455864 entries, 0 to 2455863
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   PURCHASE_DATE  object 
 1   MAGIC_KEY      object 
 2   BOX_ID         float64
 3   BOX_COUNT      float64
dtypes: float64(2), object(2)
memory usage: 74.9+ MB


In [9]:
purchase.isna().sum()

PURCHASE_DATE     0
MAGIC_KEY         0
BOX_ID           47
BOX_COUNT        47
dtype: int64

In [10]:
p1 = pd.read_csv('/kaggle/input/robi-datathon-30-qualifier-problem-1/problem 1.csv')
p1

,MAGIC_KEY
0,28D5BB06356
1,293BEAB4E98
2,2962EE8065C
3,2957BE29EA9
4,28E351A0745
...,...
58684,28FB7C09776
58685,28E0E3B69BF
58686,28D343103A7
58687,290B1D6D5CB


In [11]:
p1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58689 entries, 0 to 58688
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   MAGIC_KEY  58689 non-null  object
dtypes: object(1)
memory usage: 458.6+ KB


In [12]:
purchase['PURCHASE_DATE'] = pd.to_datetime(purchase['PURCHASE_DATE'], format='%d/%m/%Y')

In [13]:
purchase

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT
0,2019-02-01,2CED678A247,12.0,1.0
1,2019-02-01,2BF58D91BA1,12.0,1.0
2,2019-02-01,2C15B86534E,99.0,1.0
3,2019-02-01,2C32D9A859A,6.0,1.0
4,2019-02-01,2C7A55404D1,4.0,1.0
...,...,...,...,...
2455859,2018-10-28,2BD992B5538,12.0,1.0
2455860,2018-10-28,2C97CD72233,17.0,1.0
2455861,2018-10-28,2C91C61D372,40.0,1.0
2455862,2018-10-28,2CD70CFC4E3,51.0,1.0


In [14]:
merge1 = pd.merge(purchase, box, on='BOX_ID', how='left')
merge1

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,2019-02-01,2CED678A247,12.0,1.0,Premium,Home Delivery - CoD,8.0,1.5,12.98
1,2019-02-01,2BF58D91BA1,12.0,1.0,Premium,Home Delivery - CoD,8.0,1.5,12.98
2,2019-02-01,2C15B86534E,99.0,1.0,Premium,Delivery from Collection Point,0.0,3.3,13.96
3,2019-02-01,2C32D9A859A,6.0,1.0,Premium,Home Delivery - CoD,0.0,2.7,11.96
4,2019-02-01,2C7A55404D1,4.0,1.0,Premium,Home Delivery - CoD,0.0,2.5,11.96
...,...,...,...,...,...,...,...,...,...
2455859,2018-10-28,2BD992B5538,12.0,1.0,Premium,Home Delivery - CoD,8.0,1.5,12.98
2455860,2018-10-28,2C97CD72233,17.0,1.0,Premium,Home Delivery - CoD,10.0,1.8,12.98
2455861,2018-10-28,2C91C61D372,40.0,1.0,Premium,Home Delivery - CoD,12.0,1.8,19.98
2455862,2018-10-28,2CD70CFC4E3,51.0,1.0,Premium,Home Delivery - CoD,18.0,2.9,23.98


In [15]:
merge1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2455864 entries, 0 to 2455863
Data columns (total 9 columns):
 #   Column           Dtype         
---  ------           -----         
 0   PURCHASE_DATE    datetime64[ns]
 1   MAGIC_KEY        object        
 2   BOX_ID           float64       
 3   BOX_COUNT        float64       
 4   QUALITY          object        
 5   DELIVERY_OPTION  object        
 6   MILK             float64       
 7   MEAT             float64       
 8   UNIT_PRICE       float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 168.6+ MB


In [16]:
merge1.nunique()

PURCHASE_DATE          151
MAGIC_KEY          1274108
BOX_ID                 291
BOX_COUNT               14
QUALITY                  2
DELIVERY_OPTION          3
MILK                    28
MEAT                    39
UNIT_PRICE              18
dtype: int64

In [17]:
df = pd.DataFrame(merge1["MAGIC_KEY"].unique(), columns=['MAGIC_KEY'])

In [18]:
df

,MAGIC_KEY
0,2CED678A247
1,2BF58D91BA1
2,2C15B86534E
3,2C32D9A859A
4,2C7A55404D1
...,...
1274103,2BE0FCC2107
1274104,2BD992B5538
1274105,2C97CD72233
1274106,2C91C61D372


In [19]:
temp = merge1.copy()

In [20]:
temp

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,2019-02-01,2CED678A247,12.0,1.0,Premium,Home Delivery - CoD,8.0,1.5,12.98
1,2019-02-01,2BF58D91BA1,12.0,1.0,Premium,Home Delivery - CoD,8.0,1.5,12.98
2,2019-02-01,2C15B86534E,99.0,1.0,Premium,Delivery from Collection Point,0.0,3.3,13.96
3,2019-02-01,2C32D9A859A,6.0,1.0,Premium,Home Delivery - CoD,0.0,2.7,11.96
4,2019-02-01,2C7A55404D1,4.0,1.0,Premium,Home Delivery - CoD,0.0,2.5,11.96
...,...,...,...,...,...,...,...,...,...
2455859,2018-10-28,2BD992B5538,12.0,1.0,Premium,Home Delivery - CoD,8.0,1.5,12.98
2455860,2018-10-28,2C97CD72233,17.0,1.0,Premium,Home Delivery - CoD,10.0,1.8,12.98
2455861,2018-10-28,2C91C61D372,40.0,1.0,Premium,Home Delivery - CoD,12.0,1.8,19.98
2455862,2018-10-28,2CD70CFC4E3,51.0,1.0,Premium,Home Delivery - CoD,18.0,2.9,23.98


In [21]:
temp["QUALITY"].unique()

array(['Premium', 'Standard', nan], dtype=object)

In [22]:
temp["QUALITY"].value_counts()

QUALITY
Premium     1756439
Standard     699361
Name: count, dtype: int64

In [23]:
temp["QUALITY"].fillna("Other")

quality_mapping = {"Standard": 0, "Premium": 1, "Other": 0}
temp["QUALITY"] = temp["QUALITY"].map(quality_mapping)

In [24]:
temp["QUALITY"].value_counts()

QUALITY
1.0    1756439
0.0     699361
Name: count, dtype: int64

In [25]:
temp["DELIVERY_OPTION"].unique()

array(['Home Delivery - CoD', 'Delivery from Collection Point',
       'Home Delivery - Digital Payment', nan], dtype=object)

In [26]:
temp["DELIVERY_OPTION"].value_counts()

DELIVERY_OPTION
Delivery from Collection Point     2138363
Home Delivery - CoD                 314434
Home Delivery - Digital Payment       3003
Name: count, dtype: int64

In [27]:
temp["DELIVERY_OPTION"].fillna("Other")
delivery_mapping = {
    'Home Delivery - CoD': 0,
    'Delivery from Collection Point': 1,
    'Home Delivery - Digital Payment': 0,
    'Other': 0
}
temp["DELIVERY_OPTION"] = temp["DELIVERY_OPTION"].map(delivery_mapping)

In [28]:
temp

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,2019-02-01,2CED678A247,12.0,1.0,1.0,0.0,8.0,1.5,12.98
1,2019-02-01,2BF58D91BA1,12.0,1.0,1.0,0.0,8.0,1.5,12.98
2,2019-02-01,2C15B86534E,99.0,1.0,1.0,1.0,0.0,3.3,13.96
3,2019-02-01,2C32D9A859A,6.0,1.0,1.0,0.0,0.0,2.7,11.96
4,2019-02-01,2C7A55404D1,4.0,1.0,1.0,0.0,0.0,2.5,11.96
...,...,...,...,...,...,...,...,...,...
2455859,2018-10-28,2BD992B5538,12.0,1.0,1.0,0.0,8.0,1.5,12.98
2455860,2018-10-28,2C97CD72233,17.0,1.0,1.0,0.0,10.0,1.8,12.98
2455861,2018-10-28,2C91C61D372,40.0,1.0,1.0,0.0,12.0,1.8,19.98
2455862,2018-10-28,2CD70CFC4E3,51.0,1.0,1.0,0.0,18.0,2.9,23.98


In [29]:
df = pd.merge(df, temp.groupby('MAGIC_KEY')['QUALITY'].max().reset_index(), on='MAGIC_KEY', how='left')
df['Q'] = df['QUALITY']

df.drop('QUALITY', axis=1, inplace=True)
df

,MAGIC_KEY,Q
0,2CED678A247,1.0
1,2BF58D91BA1,1.0
2,2C15B86534E,1.0
3,2C32D9A859A,1.0
4,2C7A55404D1,1.0
...,...,...
1274103,2BE0FCC2107,1.0
1274104,2BD992B5538,1.0
1274105,2C97CD72233,1.0
1274106,2C91C61D372,1.0


In [30]:
df = pd.merge(df, temp.groupby('MAGIC_KEY')['DELIVERY_OPTION'].max().reset_index(), on='MAGIC_KEY', how='left')
df['D'] = df['DELIVERY_OPTION']

df.drop('DELIVERY_OPTION', axis=1, inplace=True)
df

,MAGIC_KEY,Q,D
0,2CED678A247,1.0,0.0
1,2BF58D91BA1,1.0,0.0
2,2C15B86534E,1.0,1.0
3,2C32D9A859A,1.0,0.0
4,2C7A55404D1,1.0,0.0
...,...,...,...
1274103,2BE0FCC2107,1.0,0.0
1274104,2BD992B5538,1.0,0.0
1274105,2C97CD72233,1.0,0.0
1274106,2C91C61D372,1.0,0.0


In [31]:
df["Q"].value_counts()

Q
1.0    933824
0.0    340263
Name: count, dtype: int64

In [32]:
temp1 = temp.copy()
temp1

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE
0,2019-02-01,2CED678A247,12.0,1.0,1.0,0.0,8.0,1.5,12.98
1,2019-02-01,2BF58D91BA1,12.0,1.0,1.0,0.0,8.0,1.5,12.98
2,2019-02-01,2C15B86534E,99.0,1.0,1.0,1.0,0.0,3.3,13.96
3,2019-02-01,2C32D9A859A,6.0,1.0,1.0,0.0,0.0,2.7,11.96
4,2019-02-01,2C7A55404D1,4.0,1.0,1.0,0.0,0.0,2.5,11.96
...,...,...,...,...,...,...,...,...,...
2455859,2018-10-28,2BD992B5538,12.0,1.0,1.0,0.0,8.0,1.5,12.98
2455860,2018-10-28,2C97CD72233,17.0,1.0,1.0,0.0,10.0,1.8,12.98
2455861,2018-10-28,2C91C61D372,40.0,1.0,1.0,0.0,12.0,1.8,19.98
2455862,2018-10-28,2CD70CFC4E3,51.0,1.0,1.0,0.0,18.0,2.9,23.98


In [33]:
temp1['MEAT'].fillna(0, inplace=True)

temp1['MEAT'] = np.where(temp1['MEAT'] > 0, 1, 0)
temp1['MEAT'].value_counts()

/tmp/ipykernel_33/1552129962.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temp1['MEAT'].fillna(0, inplace=True)


MEAT
1    2173448
0     282416
Name: count, dtype: int64

In [34]:
temp1['MILK'].fillna(0, inplace=True)

temp1['MILK'] = np.where(temp1['MILK'] > 0, 1, 0)
temp1['MILK'].value_counts()

/tmp/ipykernel_33/2142981455.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temp1['MILK'].fillna(0, inplace=True)


MILK
1    1748820
0     707044
Name: count, dtype: int64

In [35]:
temp1['MILK'] = (temp1['MILK'] == 1) & (temp1['PURCHASE_DATE'].dt.year == 2019) & (temp1['PURCHASE_DATE'].dt.month == 2)
temp1['MILK'] = temp1['MILK'].astype(int)
temp1['MILK'].value_counts()

MILK
0    2078649
1     377215
Name: count, dtype: int64

In [36]:
temp1['MEAT'] = (temp1['MEAT'] == 1) & (temp1['PURCHASE_DATE'].dt.year == 2019) & \
                        (temp1['PURCHASE_DATE'].dt.month == 2) & \
                        (temp1['PURCHASE_DATE'].dt.day >= 1) & \
                        (temp1['PURCHASE_DATE'].dt.day <= 15)

temp1['MEAT'] = temp1['MEAT'].astype(int)

temp1['MEAT'].value_counts()

MEAT
0    2186142
1     269722
Name: count, dtype: int64

In [37]:
dfM = df.copy()
dfM = pd.merge(dfM, temp1.groupby('MAGIC_KEY')['MILK'].max().reset_index(), on='MAGIC_KEY', how='left')
dfM

,MAGIC_KEY,Q,D,MILK
0,2CED678A247,1.0,0.0,1
1,2BF58D91BA1,1.0,0.0,1
2,2C15B86534E,1.0,1.0,0
3,2C32D9A859A,1.0,0.0,0
4,2C7A55404D1,1.0,0.0,0
...,...,...,...,...
1274103,2BE0FCC2107,1.0,0.0,0
1274104,2BD992B5538,1.0,0.0,0
1274105,2C97CD72233,1.0,0.0,0
1274106,2C91C61D372,1.0,0.0,0


In [38]:
dfM["MILK"].value_counts()

MILK
0    917256
1    356852
Name: count, dtype: int64

In [39]:
dfM = pd.merge(dfM, temp1.groupby('MAGIC_KEY')['MEAT'].max().reset_index(), on='MAGIC_KEY', how='left')
dfM

,MAGIC_KEY,Q,D,MILK,MEAT
0,2CED678A247,1.0,0.0,1,1
1,2BF58D91BA1,1.0,0.0,1,1
2,2C15B86534E,1.0,1.0,0,1
3,2C32D9A859A,1.0,0.0,0,1
4,2C7A55404D1,1.0,0.0,0,1
...,...,...,...,...,...
1274103,2BE0FCC2107,1.0,0.0,0,0
1274104,2BD992B5538,1.0,0.0,0,0
1274105,2C97CD72233,1.0,0.0,0,0
1274106,2C91C61D372,1.0,0.0,0,0


In [40]:
temp1['M15'] = (temp1['PURCHASE_DATE'].dt.day >= 1) & (temp1['PURCHASE_DATE'].dt.day <= 15)

temp1['M15'] = temp1['M15'].astype(int)
temp1['M15'].value_counts()

M15
1    1240444
0    1215420
Name: count, dtype: int64

In [41]:
dfM = pd.merge(dfM, temp1.groupby('MAGIC_KEY')['M15'].max().reset_index(), on='MAGIC_KEY', how='left')
dfM

,MAGIC_KEY,Q,D,MILK,MEAT,M15
0,2CED678A247,1.0,0.0,1,1,1
1,2BF58D91BA1,1.0,0.0,1,1,1
2,2C15B86534E,1.0,1.0,0,1,1
3,2C32D9A859A,1.0,0.0,0,1,1
4,2C7A55404D1,1.0,0.0,0,1,1
...,...,...,...,...,...,...
1274103,2BE0FCC2107,1.0,0.0,0,0,0
1274104,2BD992B5538,1.0,0.0,0,0,0
1274105,2C97CD72233,1.0,0.0,0,0,0
1274106,2C91C61D372,1.0,0.0,0,0,0


In [42]:
dfM["M15"].value_counts()

M15
1    792076
0    482032
Name: count, dtype: int64

In [43]:
df1 = dfM.copy()

In [44]:
df1

,MAGIC_KEY,Q,D,MILK,MEAT,M15
0,2CED678A247,1.0,0.0,1,1,1
1,2BF58D91BA1,1.0,0.0,1,1,1
2,2C15B86534E,1.0,1.0,0,1,1
3,2C32D9A859A,1.0,0.0,0,1,1
4,2C7A55404D1,1.0,0.0,0,1,1
...,...,...,...,...,...,...
1274103,2BE0FCC2107,1.0,0.0,0,0,0
1274104,2BD992B5538,1.0,0.0,0,0,0
1274105,2C97CD72233,1.0,0.0,0,0,0
1274106,2C91C61D372,1.0,0.0,0,0,0


In [45]:
temp2 = temp1.copy()

In [46]:
temp2

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,M15
0,2019-02-01,2CED678A247,12.0,1.0,1.0,0.0,1,1,12.98,1
1,2019-02-01,2BF58D91BA1,12.0,1.0,1.0,0.0,1,1,12.98,1
2,2019-02-01,2C15B86534E,99.0,1.0,1.0,1.0,0,1,13.96,1
3,2019-02-01,2C32D9A859A,6.0,1.0,1.0,0.0,0,1,11.96,1
4,2019-02-01,2C7A55404D1,4.0,1.0,1.0,0.0,0,1,11.96,1
...,...,...,...,...,...,...,...,...,...,...
2455859,2018-10-28,2BD992B5538,12.0,1.0,1.0,0.0,0,0,12.98,0
2455860,2018-10-28,2C97CD72233,17.0,1.0,1.0,0.0,0,0,12.98,0
2455861,2018-10-28,2C91C61D372,40.0,1.0,1.0,0.0,0,0,19.98,0
2455862,2018-10-28,2CD70CFC4E3,51.0,1.0,1.0,0.0,0,0,23.98,0


In [47]:
a = temp2["BOX_COUNT"].fillna(1)
temp2["BOX_COUNT"] = a
temp2["BOX_COUNT"] = temp2["BOX_COUNT"].replace(-1, 1)
temp2["BOX_COUNT"].unique()

array([ 1.,  2.,  4.,  3.,  8.,  6.,  5.,  7.,  9., 19., 10., 11., 13.])

In [48]:
a = temp2["UNIT_PRICE"].fillna(0)
temp2["UNIT_PRICE"] = a
temp2["UNIT_PRICE"].unique()

array([12.98, 13.96, 11.96, 12.78,  9.96, 10.14, 12.  , 15.96, 15.98,
       23.98, 12.18, 19.98, 17.98,  0.  ,  9.98, 15.  , 12.96,  9.94,
        5.56])

In [49]:
temp2["PRICE"] = temp2["BOX_COUNT"] * temp2["UNIT_PRICE"]
temp2

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,M15,PRICE
0,2019-02-01,2CED678A247,12.0,1.0,1.0,0.0,1,1,12.98,1,12.98
1,2019-02-01,2BF58D91BA1,12.0,1.0,1.0,0.0,1,1,12.98,1,12.98
2,2019-02-01,2C15B86534E,99.0,1.0,1.0,1.0,0,1,13.96,1,13.96
3,2019-02-01,2C32D9A859A,6.0,1.0,1.0,0.0,0,1,11.96,1,11.96
4,2019-02-01,2C7A55404D1,4.0,1.0,1.0,0.0,0,1,11.96,1,11.96
...,...,...,...,...,...,...,...,...,...,...,...
2455859,2018-10-28,2BD992B5538,12.0,1.0,1.0,0.0,0,0,12.98,0,12.98
2455860,2018-10-28,2C97CD72233,17.0,1.0,1.0,0.0,0,0,12.98,0,12.98
2455861,2018-10-28,2C91C61D372,40.0,1.0,1.0,0.0,0,0,19.98,0,19.98
2455862,2018-10-28,2CD70CFC4E3,51.0,1.0,1.0,0.0,0,0,23.98,0,23.98


In [50]:
dfM = pd.merge(dfM, temp2.groupby('MAGIC_KEY')['PRICE'].mean().reset_index(), on='MAGIC_KEY', how='left')
dfM = pd.merge(dfM, temp2.groupby('MAGIC_KEY')['BOX_COUNT'].mean().reset_index(), on='MAGIC_KEY', how='left')

dfM

,MAGIC_KEY,Q,D,MILK,MEAT,M15,PRICE,BOX_COUNT
0,2CED678A247,1.0,0.0,1,1,1,12.98,1.0
1,2BF58D91BA1,1.0,0.0,1,1,1,12.98,1.0
2,2C15B86534E,1.0,1.0,0,1,1,12.96,1.0
3,2C32D9A859A,1.0,0.0,0,1,1,11.96,1.0
4,2C7A55404D1,1.0,0.0,0,1,1,11.96,1.0
...,...,...,...,...,...,...,...,...
1274103,2BE0FCC2107,1.0,0.0,0,0,0,12.98,1.0
1274104,2BD992B5538,1.0,0.0,0,0,0,12.98,1.0
1274105,2C97CD72233,1.0,0.0,0,0,0,12.98,1.0
1274106,2C91C61D372,1.0,0.0,0,0,0,19.98,1.0


In [51]:
dfM['P'] = (dfM['PRICE'] / dfM["BOX_COUNT"])
dfM.drop('PRICE', axis=1, inplace=True)
dfM.drop('BOX_COUNT', axis=1, inplace=True)

dfM

,MAGIC_KEY,Q,D,MILK,MEAT,M15,P
0,2CED678A247,1.0,0.0,1,1,1,12.98
1,2BF58D91BA1,1.0,0.0,1,1,1,12.98
2,2C15B86534E,1.0,1.0,0,1,1,12.96
3,2C32D9A859A,1.0,0.0,0,1,1,11.96
4,2C7A55404D1,1.0,0.0,0,1,1,11.96
...,...,...,...,...,...,...,...
1274103,2BE0FCC2107,1.0,0.0,0,0,0,12.98
1274104,2BD992B5538,1.0,0.0,0,0,0,12.98
1274105,2C97CD72233,1.0,0.0,0,0,0,12.98
1274106,2C91C61D372,1.0,0.0,0,0,0,19.98


In [52]:
dfM["P"].max(), dfM["P"].min(), dfM["P"].mean(), dfM["P"].median()

(23.980000000000004, 0.0, 17.205502129272656, 15.98)

In [53]:
dfM["P"].nunique()

3469

In [54]:
mean_P = dfM['P'].mean()

dfM['P'] = (dfM['P'] > 0) & (dfM['P'] < mean_P)

dfM['P'] = dfM['P'].astype(int)
dfM['P'].value_counts()

P
1    712558
0    561550
Name: count, dtype: int64

In [55]:
dfM

,MAGIC_KEY,Q,D,MILK,MEAT,M15,P
0,2CED678A247,1.0,0.0,1,1,1,1
1,2BF58D91BA1,1.0,0.0,1,1,1,1
2,2C15B86534E,1.0,1.0,0,1,1,1
3,2C32D9A859A,1.0,0.0,0,1,1,1
4,2C7A55404D1,1.0,0.0,0,1,1,1
...,...,...,...,...,...,...,...
1274103,2BE0FCC2107,1.0,0.0,0,0,0,1
1274104,2BD992B5538,1.0,0.0,0,0,0,1
1274105,2C97CD72233,1.0,0.0,0,0,0,1
1274106,2C91C61D372,1.0,0.0,0,0,0,0


In [56]:
df2 = dfM.copy()
temp3 = temp2.copy()

In [57]:
df2

,MAGIC_KEY,Q,D,MILK,MEAT,M15,P
0,2CED678A247,1.0,0.0,1,1,1,1
1,2BF58D91BA1,1.0,0.0,1,1,1,1
2,2C15B86534E,1.0,1.0,0,1,1,1
3,2C32D9A859A,1.0,0.0,0,1,1,1
4,2C7A55404D1,1.0,0.0,0,1,1,1
...,...,...,...,...,...,...,...
1274103,2BE0FCC2107,1.0,0.0,0,0,0,1
1274104,2BD992B5538,1.0,0.0,0,0,0,1
1274105,2C97CD72233,1.0,0.0,0,0,0,1
1274106,2C91C61D372,1.0,0.0,0,0,0,0


In [58]:
temp3

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,M15,PRICE
0,2019-02-01,2CED678A247,12.0,1.0,1.0,0.0,1,1,12.98,1,12.98
1,2019-02-01,2BF58D91BA1,12.0,1.0,1.0,0.0,1,1,12.98,1,12.98
2,2019-02-01,2C15B86534E,99.0,1.0,1.0,1.0,0,1,13.96,1,13.96
3,2019-02-01,2C32D9A859A,6.0,1.0,1.0,0.0,0,1,11.96,1,11.96
4,2019-02-01,2C7A55404D1,4.0,1.0,1.0,0.0,0,1,11.96,1,11.96
...,...,...,...,...,...,...,...,...,...,...,...
2455859,2018-10-28,2BD992B5538,12.0,1.0,1.0,0.0,0,0,12.98,0,12.98
2455860,2018-10-28,2C97CD72233,17.0,1.0,1.0,0.0,0,0,12.98,0,12.98
2455861,2018-10-28,2C91C61D372,40.0,1.0,1.0,0.0,0,0,19.98,0,19.98
2455862,2018-10-28,2CD70CFC4E3,51.0,1.0,1.0,0.0,0,0,23.98,0,23.98


In [59]:
temp3['F15'] = 0
temp3.loc[(temp3['PURCHASE_DATE'] >= '2019-02-01') & (temp3['PURCHASE_DATE'] <= '2019-02-15'), 'F15'] = 2
temp3['F15'].value_counts()

F15
0    2158799
2     297065
Name: count, dtype: int64

In [60]:
dfM = pd.merge(dfM, temp3.groupby('MAGIC_KEY')['F15'].max().reset_index(), on='MAGIC_KEY', how='left')
dfM

,MAGIC_KEY,Q,D,MILK,MEAT,M15,P,F15
0,2CED678A247,1.0,0.0,1,1,1,1,2
1,2BF58D91BA1,1.0,0.0,1,1,1,1,2
2,2C15B86534E,1.0,1.0,0,1,1,1,2
3,2C32D9A859A,1.0,0.0,0,1,1,1,2
4,2C7A55404D1,1.0,0.0,0,1,1,1,2
...,...,...,...,...,...,...,...,...
1274103,2BE0FCC2107,1.0,0.0,0,0,0,1,0
1274104,2BD992B5538,1.0,0.0,0,0,0,1,0
1274105,2C97CD72233,1.0,0.0,0,0,0,1,0
1274106,2C91C61D372,1.0,0.0,0,0,0,0,0


In [61]:
dfM["F15"].value_counts()

F15
0    979795
2    294313
Name: count, dtype: int64

In [62]:
df3 = dfM.copy()

In [63]:
dfM["F"] = dfM["MAGIC_KEY"].map(temp3["MAGIC_KEY"].value_counts())
dfM["F"].value_counts()

F
1     719534
2     248022
3     132665
4      84301
5      58552
6      19873
7       5655
8       2376
9       1216
10       704
11       376
12       248
13       178
14       119
15        70
16        50
17        32
18        31
19        18
20        16
23        13
22        13
21        12
25         7
24         6
26         5
29         4
35         2
31         2
34         2
30         1
36         1
38         1
32         1
40         1
28         1
Name: count, dtype: int64

In [64]:
dfM['F'] = np.where(dfM['F'] > 1, 3, 0)
dfM["F"].value_counts()

F
0    719534
3    554574
Name: count, dtype: int64

In [65]:
train = dfM.copy()
train

,MAGIC_KEY,Q,D,MILK,MEAT,M15,P,F15,F
0,2CED678A247,1.0,0.0,1,1,1,1,2,0
1,2BF58D91BA1,1.0,0.0,1,1,1,1,2,0
2,2C15B86534E,1.0,1.0,0,1,1,1,2,3
3,2C32D9A859A,1.0,0.0,0,1,1,1,2,0
4,2C7A55404D1,1.0,0.0,0,1,1,1,2,0
...,...,...,...,...,...,...,...,...,...
1274103,2BE0FCC2107,1.0,0.0,0,0,0,1,0,0
1274104,2BD992B5538,1.0,0.0,0,0,0,1,0,0
1274105,2C97CD72233,1.0,0.0,0,0,0,1,0,0
1274106,2C91C61D372,1.0,0.0,0,0,0,0,0,0


In [66]:
train['PURCHASE'] = train['Q'] + train['D'] + train['MILK'] + train['MEAT'] + train['M15'] + train['P'] + train['F15'] + train['F']

train['PURCHASE'] = train['PURCHASE'].apply(lambda x: 'Y' if x > 8 else 'N')
train["PURCHASE"].value_counts()

PURCHASE
N    1089948
Y     184160
Name: count, dtype: int64

In [68]:
test = p1.copy()

In [69]:
test = test.merge(train[['MAGIC_KEY', 'PURCHASE']], on='MAGIC_KEY', how='left')
test["PURCHASE"].value_counts()

PURCHASE
N    29946
Y    28743
Name: count, dtype: int64

In [70]:
test

,MAGIC_KEY,PURCHASE
0,28D5BB06356,N
1,293BEAB4E98,Y
2,2962EE8065C,N
3,2957BE29EA9,N
4,28E351A0745,N
...,...,...
58684,28FB7C09776,N
58685,28E0E3B69BF,N
58686,28D343103A7,N
58687,290B1D6D5CB,Y


In [71]:
test.to_csv("submission.csv", columns=['MAGIC_KEY', 'PURCHASE'], index=False)